# Part 1 - Fundamentals of Quantum Computing

by **Francesco Tacchino** and **Chiara Macchiavello** 
-- review for UniMI Workshop by **Michele Grossi** 

<p style='text-align: justify;'> In this chapter, we will present and discuss many basic concepts about quantum circuits and quantum algorithms. We will see how to implement and verify quantum gates and circuit identities in qiskit. Many of the codes in this tutorial can be adapted to run on IBMQ quantum devices with the methods presented in [Part 0-Introduction](./Introduction.ipynb).</p>

## Bits, Qubits and Circuits

<p style='text-align: justify;'> Quantum computation, similarly to its classical counterpart, is often described and discussed in terms of a circuital model. Information carriers are called _quantum bits_ (_qubits_ in short) and will be represented with straight lines. The flow of information is controlled by single- or multi-qubit operations known as _quantum gates_ which are represented as boxes with input and output qubit wires. During the quantum computation and at the end of it, classical information, including for example the memory slots in which the results of specific operations are stored, is carried around by classical bits, represented by double straight lines. We have already seen some examples of _quantum circuits_, i.e. collections of qubits (i.e. _quantum registers_), _classical registers_ of bits and operations, in [Part 0-Introduction](./Introduction.ipynb), but we will present here their properties in greater detail. In general terms, there are three fundamental stages common to any quantum computation protocol:</p>

* <p style='text-align: justify;'> __input preparation__, i.e. encoding of the initial information into the quantum state of $N$ qubits;</p>
* <p style='text-align: justify;'> __processing of the information__, which is carried out by controlling the physical evolution of the qubits, with quantum gates corresponding to unitary transformations;</p>
* <p style='text-align: justify;'>__output readout__, which is performed by measuring (in the quantum mechanical sense) some specific properties of the qubits. Since quantum measurements are intrinsically probabilistic, the same is also true for many quantum algorithms.</p>

<p style='text-align: justify;'> In qiskit we can set the stage with one qubit and one classical bit as follows:</p>

In [ ]:
import qiskit as qk

qr0 = qk.QuantumRegister(1,name='qb')  # qb for "qubit", but the name is optional
cr0 = qk.ClassicalRegister(1,name='b') # b for "bit", but the name is optional
qc0 = qk.QuantumCircuit(qr0,cr0)


qc0.draw(output='latex') # this is to visualize the circuit. Use 'mpl' or no output option for different styles.
                         # Notice that mpl could fail with an empty circuit like this one.

<p style='text-align: justify;'> In qiskit all qubits always start from the blank state $\left|0\right\rangle$. In the next section, we will learn more about qubits and qubit states. </p>

### Qubits and the Bloch Sphere

<p style='text-align: justify;'> Any quantum mechanical system with an associated Hilbert space $\mathcal{H}$ of dimension $d = 2$ can represent an elementary quantum information carrier (qubit). Very well known examples are spin $s = 1/2$ particles, even though qubits in modern quantum processors are realized with a variety of physical systems such as superconducting circuits or trapped ions. The canonical basis for $\mathcal{H}$ is called, in quantum information jargon, the _computational basis_ and is represented as follows:
$$
\left|0\right\rangle = \begin{pmatrix}
1 \\
0
\end{pmatrix} \qquad
\left|1\right\rangle = \begin{pmatrix}
0 \\
1
\end{pmatrix}
$$
Any generic state of a qubit can therefore be written as a linear combination of the above basis states
$$
\left|\psi\right\rangle = \alpha\left|0\right\rangle + \beta\left|1\right\rangle
$$
with $\alpha,\beta \in \mathbb{C}$ and $\left|\alpha\right|^2 + \left|\beta\right|^2$. The latter is known as the _normalization_ condition. Any state $\left|\psi\right\rangle$ can be conveniently rewritten with two real parameters $\theta$ and $\phi$ as follows:
$$
\left|\psi(\theta,\phi)\right\rangle = \cos\frac{\theta}{2}\left|0\right\rangle + e^{i\phi}\sin\frac{\theta}{2}\left|1\right\rangle
$$
with $0\leq\theta\leq\pi$ and $0\leq\phi\leq 2\pi$. As a result, any quantum state can be interpreted graphically as a point on the surface of a 3d unit (i.e. with radius 1) sphere known as the _Bloch sphere_. The cartesian coordinates of such point are:
$$
x = \cos\phi\sin\theta
$$
$$
y = \sin\phi\sin\theta
$$
$$
z = \cos\theta
$$
where $\theta$ plays the role of the polar angle and $\phi$ of the azimuthal angle in a sperical coordinate system. Using this convention, the general state vector becomes
$$
\left|\psi(\theta,\phi)\right\rangle = \begin{pmatrix}
\cos\frac{\theta}{2} \\
e^{i\phi}\sin\frac{\theta}{2}
\end{pmatrix}
$$
and the the basis states $\left|0\right\rangle$ and $\left|1\right\rangle$ are placed on the north and south pole of the Bloch sphere respectively.

In qiskit, we can directly plot the vector representing a quantum state on the Bloch sphere using the following instructions:</p>

In [ ]:
from qiskit.tools.visualization import plot_bloch_vector
import math

theta = math.pi/2    # You can try to change the parameters theta and phi
phi = math.pi/4      # to see how the vector moves on the sphere
plot_bloch_vector([math.cos(phi)*math.sin(theta),math.sin(phi)*math.sin(theta),math.cos(theta)])  # Entries of the 
                                                                                                  # input vector are 
                                                                                                  # coordinates [x,y,z]

<p style='text-align: justify;'> Later we will see how to plot directly on a Bloch sphere the output of a quantum circuit, a feature which might be useful for visualizing the action of individual gates and for debugging. </p>

* **Exercise 1.1** Compute (with pen and paper!) the eigenstates of single-qubit Pauli operators 
$$\sigma_x = \begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix} \qquad
\sigma_y = \begin{pmatrix}
0 & -i \\
i & 0
\end{pmatrix} \qquad
\sigma_z = \begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}$$
<p style='text-align: justify;'> Find the corresponding parameters $\theta$ and $\phi$ for all eigenstates and represent them on the Bloch sphere using qiskit. How do they align with the coordinate axes?</p>

## Single qubit operations

<p style='text-align: justify;'> Now that we have a mathematical description of an individual qubit, we can start performing operations on it. In the following, we will describe some of the unitary gates which are of major interest for quantum computing applications. Some of such gates are directly available in qiskit, while others must be reconstructed by combinations of the available operations.</p>

#### Measurement in the computational basis

<p style='text-align: justify;'> Before diving into unitary quantum gates, let us spend a few words about the ways we can access the information stored in a qubit register when using qiskit. At the end of a cicruit, the state on any qubit can be read to reveal whether, for example, the qubit itself is found in $\left|0\right\rangle$ or $\left|1\right\rangle$. This is known as a _measure in the computational basis_. When the state $\left|\psi\right\rangle = \alpha\left|0\right\rangle + \beta\left|1\right\rangle$ contains both $\left|0\right\rangle$ and $\left|1\right\rangle$ components, the outcome of a single measure is probabilistic, and the two results are observed with probabilities $p_0 = \left|\alpha\right|^2$ and $p_1 = \left|\beta\right|^2$. If the same circuit is repeated several times (the keyword option in qiskit is `shots`), then the statistics of outcomes can easily be reconstructed. As an example, we can add a measure to the circuit `qc0`, simulate it 1024 times and see how many times the classical bit on which we store the result is 0, given that the state on the qubit is $\left|0\right\rangle$ </p>

In [ ]:
qc0.measure(qr0[0],cr0[0])

qc0.draw(output='mpl')

In [ ]:
backend = qk.BasicAer.get_backend('qasm_simulator')
job = qk.execute(qc0, backend, shots=1024)
result = job.result()
counts = result.get_counts()
print(counts)

#### The $\mathrm{NOT}$  (or $\mathrm{X}$) gate

<p style='text-align: justify;'> This gate is the exact quantum counterpart of the classical bit negation operation. As a single qubit unitary operation, it is represented by the matrix
$$
\mathrm{NOT} = \begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}
$$
acting as follows on the computational basis:
$$
\left|0\right\rangle \rightarrow \left|1\right\rangle \qquad \left|1\right\rangle \rightarrow \left|0\right\rangle  
$$
Since the $\mathrm{NOT}$ matrix is exactly identical to the $\sigma_x$ Pauli matrix, this gate is also ofter denoted with $\mathrm{X}$. In qiskit, the gate is available with the name `x`. We can have a look at its action by comparing the results of the circuits `qc0` (seen above) and `qc1` (defined below):</p>

In [ ]:
qc1 = qk.QuantumCircuit(qr0,cr0)
qc1.x(qr0[0]) # A X gate targeting the first (and only) qubit in register qr0
qc1.measure(qr0[0],cr0[0])

qc1.draw(output='mpl')

In [ ]:
backend = qk.BasicAer.get_backend('qasm_simulator')
job1 = qk.execute(qc1, backend, shots=1024)
result1 = job1.result()
counts1 = result1.get_counts()
print(counts1)

<p style='text-align: justify;'> As expected, the $\mathrm{X}$ gate brings the initial state $\left|0\right\rangle$ into the state $\left|1\right\rangle$. We can plot the same action on the bloch sphere using the backend called `statevector_simulator`:</p>

In [ ]:
from qiskit.tools.visualization import plot_bloch_multivector

vec_backend = qk.BasicAer.get_backend('statevector_simulator')
result_vec0 = qk.execute(qc0, vec_backend).result()
result_vec1 = qk.execute(qc1, vec_backend).result()
psi0 = result_vec0.get_statevector()
psi1 = result_vec1.get_statevector()

In [ ]:
plot_bloch_multivector(psi0)

In [ ]:
plot_bloch_multivector(psi1)

<p style='text-align: justify;'> When applied to a superposition of basis states, the action of the gate, as for any other quantum operation, extends linearly:
$$
\mathrm{X}\left(\alpha\left|0\right\rangle + \beta\left|1\right\rangle\right) = \alpha\left|1\right\rangle + \beta\left|0\right\rangle
$$

In geometrical terms, the $\mathrm{X}$ gate is a rotation of the state vector (as represented on the Bloch sphere) by an angle $\pi$ around the $x$ axis.</p>

#### The Hadamard gate

<p style='text-align: justify;'> This gate is ubiquitous in many quantum algorithms and has no classical counterpart, since it can bring a qubit from the computational basis to a quantum superposition of states. The gate is denoted by $\mathrm{H}$ and is represented by the matrix
$$
\mathrm{H} = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix}
$$
It acts on the computational basis as follows:
$$
\left|0\right\rangle \rightarrow \frac{1}{\sqrt{2}}\left(\left|0\right\rangle + \left|1\right\rangle\right) \qquad \left|1\right\rangle \rightarrow \frac{1}{\sqrt{2}}\left(\left|0\right\rangle - \left|1\right\rangle\right)
$$

If we put it in a quantum circuit starting from a computational basis state, we get an equal (up to statistical errors) distribution of 0 and 1 outcomes:</p>

In [ ]:
qc2 = qk.QuantumCircuit(qr0,cr0)
qc2.h(qr0[0])
qc2.measure(qr0[0],cr0[0])

qc2.draw(output='mpl')

In [ ]:
job2 = qk.execute(qc2, backend, shots=1024)
result2 = job2.result()
counts2 = result2.get_counts()
print(counts2)

In [ ]:
result_vec2 = qk.execute(qc2, vec_backend).result()
psi2 = result_vec2.get_statevector()
plot_bloch_multivector(psi2)

<p style='text-align: justify;'> In geometrical terms, the $\mathrm{H}$ gate is a rotation of the state vector by an angle $\pi/2$ around the axis identified by the linear relation $z=x$. Notice that applyng $\mathrm{H}$ twice brings the qubit back to its original state, i.e. $\mathrm{H}^2 = \mathcal{I}$, where $\mathcal{I}$ is the identity matrix.</p>

#### Other qiskit single qubit gates

<p style='text-align: justify;'> In qiskit, there are a few other standard quantum gates that you can code straight away.</p>

<p style='text-align: justify;'> __The $\mathbf{\mathrm{S}}$ gate.__ This gate has the following unitary matrix
$$
\mathrm{S} = \begin{pmatrix}
1 & 0 \\
0 & i
\end{pmatrix}
$$
and, geometrically speaking, it rotates the state vector by an angle $\pi/2$ around the $z$ axis. Its adjoint (i.e. conjugate transpose)
$$
\mathrm{S}^\dagger = \begin{pmatrix}
1 & 0 \\
0 & -i
\end{pmatrix}
$$
rotates by an angle $-\pi/2$ instead. The qiskit syntax is the following (we use a Hadamard gate first to put the state vector on the $xy$ plane)</p>

In [ ]:
qc3 = qk.QuantumCircuit(qr0,cr0)
qc3.h(qr0[0])
qc3.s(qr0[0]) # Use sdg instead of s for the adjoint
qc3.measure(qr0[0],cr0[0])

qc3.draw(output='mpl')

In [ ]:
job3 = qk.execute(qc3, backend, shots=1024)
result3 = job3.result()
counts3 = result3.get_counts()
print(counts3)

In [ ]:
result_vec3 = qk.execute(qc3, vec_backend).result()
psi3 = result_vec3.get_statevector()
plot_bloch_multivector(psi3)

<p style='text-align: justify;'> __The $\mathbf{\mathrm{T}}$ gate.__ This gate has the following unitary matrix
$$
\mathrm{T} = \begin{pmatrix}
1 & 0 \\
0 & (1+i)/\sqrt{2}
\end{pmatrix}
$$
and, geometrically speaking, it rotates the state vector by an angle $\pi/4$ around the $z$ axis. Its adjoint (i.e. conjugate transpose)
$$
\mathrm{T}^\dagger = \begin{pmatrix}
1 & 0 \\
0 & (1-i)/\sqrt{2}
\end{pmatrix}
$$
rotates by an angle $-\pi/4$ instead. The qiskit syntax is the following (again, we use a Hadamard gate first to put the state vector on the $xy$ plane)</p>

In [ ]:
qc4 = qk.QuantumCircuit(qr0,cr0)
qc4.h(qr0[0])
qc4.t(qr0[0]) # Use tdg instead of t for the adjoint
qc4.measure(qr0[0],cr0[0])

qc4.draw(output='mpl')

In [ ]:
job4 = qk.execute(qc4, backend, shots=1024)
result4 = job4.result()
counts4 = result4.get_counts()
print(counts4)

In [ ]:
result_vec4 = qk.execute(qc4, vec_backend).result()
psi4 = result_vec4.get_statevector()
plot_bloch_multivector(psi4)

<p style='text-align: justify;'> __The Pauli gates.__ The Pauli gates $\mathrm{X}$, $\mathrm{Y}$ and $\mathrm{Z}$ are direct representations of the Pauli matrices
$$
\mathrm{X} = \sigma_x \qquad \mathrm{Y} = \sigma_y \qquad \mathrm{Z} = \sigma_z 
$$
and act on a qubit by rotating its state vector by an angle $\pi$ around their corresponding axis. For example, we have already seen how $\mathrm{NOT} \equiv \mathrm{X}$ is a $\pi$ rotation around the $x$ direction. They can be coded in qiskit using the `.x`, `.y` and `.z` syntax.</p>

<p style='text-align: justify;'> __IBMQ native single qubit gates.__ On IBMQ real processors, all single qubit operations are performed at the hardware level as combinations of three native $\mathrm{U}_i$ operations, whihch depend on continuous parameters. These gates are directly available in qiskit, and all others (e.g. $\mathrm{X}$, $\mathrm{H}$,...) must be translated into (combinations of) $\mathrm{U}_i$s during the compilation stage before they can be run on the IBMQ devices. In this sense, the syntaxes for the gates that we have seen above are essentially shorthands for combinations of native gates. 

The first native gate $\mathrm{U}_1$ represents the so called _phase_ gate, i.e. a rotation of an arbitrary angle $\lambda$ around the $z$ axis:
$$
\mathrm{U}_1 (\lambda) = \begin{pmatrix}
1 & 0 \\
0 & e^{i\lambda}
\end{pmatrix}
$$
As an example, we can use it in combination with an $\mathrm{H}$ gate to reach any point on the equator of the Bloch sphere:</p>

In [ ]:
lmbd = 1.2  # Change this value to rotate the output vector on the equator of the Bloch sphere
qc5 = qk.QuantumCircuit(qr0,cr0)
qc5.h(qr0[0])
qc5.u1(lmbd, qr0[0])
qc5.measure(qr0[0],cr0[0])

qc5.draw(output='mpl')

In [ ]:
result_vec5 = qk.execute(qc5, vec_backend).result()
psi5 = result_vec5.get_statevector()
plot_bloch_multivector(psi5)

<p style='text-align: justify;'> The $\mathrm{U}_2$ gate adds one degree of freedom with an additional parameter:
$$
\mathrm{U}_2 (\phi,\lambda) = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & -e^{i\lambda} \\
e^{i\phi} & e^{i(\lambda+\phi)}
\end{pmatrix}
$$
It can for example directly reproduce the $\mathrm{H}$ gate, since $\mathrm{H}=\mathrm{U}_2(0,\pi)$. We can see it using qiskit:</p>

In [ ]:
qc6 = qk.QuantumCircuit(qr0,cr0)
qc6.u2(0,math.pi, qr0[0])
qc6.measure(qr0[0],cr0[0])

result_vec6 = qk.execute(qc6, vec_backend).result()
psi6 = result_vec6.get_statevector()
plot_bloch_multivector(psi6)

<p style='text-align: justify;'> Finally, the $\mathrm{U}_3$ gate represents the most general single qubit unitary operation and can bring the state vector from any point to any other on the Bloch sphere:
$$
\mathrm{U}_3 (\theta,\lambda,\phi) = \begin{pmatrix}
\cos(\theta/2) & -\sin(\theta/2)e^{i\lambda} \\
\sin(\theta/2)e^{i\phi} & \cos(\theta/2)e^{i(\lambda+\phi)}
\end{pmatrix}
$$
You can play with the parameters in the code below to get familiar with the action of $\mathrm{U}_3$:</p>

In [ ]:
theta = 0.5
phi = math.pi/4
lmb = math.pi/8

qc7 = qk.QuantumCircuit(qr0,cr0)
qc7.u3(theta,phi, lmb, qr0[0])
qc7.measure(qr0[0],cr0[0])

result_vec7 = qk.execute(qc7, vec_backend).result()
psi7 = result_vec7.get_statevector()
plot_bloch_multivector(psi7)

* <p style='text-align: justify;'> __Exercise 1.2__ For any of the gates described above, find an equivalent circuit containing only one or more $\mathrm{U}_i$ gates. Test your predictions using qiskit.</p>
* <p style='text-align: justify;'> __Exercise 1.3 (Qubit rotations)__ A rotation by an angle $\theta$ of a single qubit around one of the coordinate axes can be implemented with one of the following unitary operations:
$$
\mathrm{R}_x(\theta) = \exp\left(-i\theta\frac{\sigma_x}{2}\right)
$$
for a rotation around the $x$ direction,
$$
\mathrm{R}_y(\theta) = \exp\left(-i\theta\frac{\sigma_y}{2}\right)
$$
for a rotation around the $y$ direction, and
$$
\mathrm{R}_z(\theta) = \exp\left(-i\theta\frac{\sigma_z}{2}\right)
$$
for a rotation around the $z$ direction. After computing the explicit form of the corresponding $2\times2$ matrix, find an equivalent expression of these rotations in terms of $\mathrm{U}_i$ gates and test them in qiskit.</p>
* <p style='text-align: justify;'> __Exercise 1.4 (Hadamard + phase are universal for single qubit operations)__ By explicitly performing the calculations, prove that any single qubit operation can be obtained by choosing an appropriate value for the phases in the following circuit:
$$
\mathrm{U}_{circ} = \mathrm{U}_{1}(\pi/2 + \phi_2)\mathrm{H}\mathrm{U}_{1}(\theta_2-\theta_1)\mathrm{H}\mathrm{U}_{1}(- \pi/2 - \phi_1)  
$$
(_hint_: the $\mathrm{U}_3$ matrix is the most general single qubit operation, so you only need to show that the unitary transformation $\mathrm{U}_{circ}$ above is equivalent to $\mathrm{U}_3$, possibly up to a global phase. Alternatively, you can show that $\left|\psi(\theta_2,\phi_2)\right\rangle = \mathrm{U}_{circ}\left|\psi(\theta_1,\phi_1)\right\rangle\,\forall \theta_1,\theta_2,\phi_1,\phi_2$ ). Test your predictions with qiskit, paying attention to the order of unitaries in a product with respect to the corresponding circuital representation.</p>

## Two qubit gates

<p style='text-align: justify;'> In order for the quantum computational model to be complete, single qubit transformations are not sufficient. However, it can be shown that _any_ arbitrary operation on $N$ qubits can be decomposed in single- and two-qubit operations. Moreover, there exist finite sets of single- and  two-qubit gates that are universal, meaning that the gates belonging to the set can be combined to obtain any $N$-qubit operation. Some of such universal sets contain gate types that depend on a continuous parameter, such as the phase gate, but examples of sets containing only fixed-parameters gates that can approximate any arbitrary quantum operation are also known. Once and for all, we must point out that the set of gates available in qiskit (including two-qubit operations) and native to IBMQ platforms is indeed universal, since it contains a general single qubit operation (see $\mathrm{U}_3$ above) and one entangling two-qubit gate (the $\mathrm{CNOT}$, see below).</p>
  
<p style='text-align: justify;'> All universal sets contain at least one two-qubit operation which is able to generate quantum entanglement starting from a factorized state. Indeed, single qubit operations, even if applied in parallel on multiple qubits, always produce factorized states if the input is also factorized
$$
\left(\mathrm{U}_A\otimes\mathrm{U}_B \right)\left|\psi_A\right\rangle\left|\psi_B\right\rangle = \left(\mathrm{U}_A\left|\psi_A\right\rangle\right)\otimes\left(\mathrm{U}_B\left|\psi_B\right\rangle\right)
$$
and therefore a collective gate is needed to generate non-trivial quantum correlations. Here we used the tensor product notation $\otimes$ to denote the composition of two smaller systems into a larger one. As it is well known, such rule is a foundational characteristic of the mathematical structure of quantum mechanics. The tensor product symbol is often omitted between states (e.g. $\left|\psi_A\right\rangle\left|\psi_B\right\rangle \equiv \left|\psi_A\right\rangle\otimes\left|\psi_B\right\rangle$).</p>

#### Two qubits is better than one

<p style='text-align: justify;'> The computational basis for a pair of qubits is constructed starting from indivdual single-qubit computational basis states combined with a tensor product. Here we report the standard notation and the corresponding vector representation:
$$
\left|00\right\rangle =  \begin{pmatrix}
1 \\
0 \\
0 \\
0
\end{pmatrix} \qquad
\left|01\right\rangle =  \begin{pmatrix}
0 \\
1 \\
0 \\
0
\end{pmatrix}\qquad
\left|10\right\rangle =  \begin{pmatrix}
0 \\
0 \\
1 \\
0
\end{pmatrix}\qquad
\left|11\right\rangle =  \begin{pmatrix}
0 \\
0 \\
0 \\
1
\end{pmatrix}
$$
where $\left|ab\right\rangle \equiv \left|a\right\rangle\left|b\right\rangle$. It is immediately clear that the Hilbert space for two qubits has dimension $d = 2^2 = 4$.
</p>

#### The $\mathrm{CNOT}\equiv\mathrm{CX}$ gate

<p style='text-align: justify;'> This two-qubit gate is widely used in quantum computation, as it is entangling and, when complemented with single qubit operations, universal. In qiskit, it is directly available both in the programming language itself and on the IBMQ hardware, where it represents the only _native_ two-qubit interaction to which all others must be reduced (at least at the compiling stage). The $\mathrm{CNOT}$ is represented by the following matrix
$$
\mathrm{CNOT} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{pmatrix}
$$
It acts on the computational basis by performing a negation (a $\mathrm{X}$ gate in quantum computing, hence the name $\mathrm{CX}$ which is also associated to this gate) on the second qubit (_target_) controlled by the state if the first one (_control_). Using modulo 2 addition between 0 and 1 values, we can indeed write
$$
\mathrm{CNOT}\left|x\right\rangle\left|y\right\rangle = \left|x\right\rangle\left|x\oplus y\right\rangle \qquad x,y \in \{0,1\}
$$
Here is an example in qiskit that, starting from the blank state $\left|00\right\rangle$, first brings the control qubit in $\left|1\right\rangle$ with an $\mathrm{X}$ gate, and then performs a $\mathrm{CNOT}$ operation to flip also the target qubit.</p>

In [ ]:
qr = qk.QuantumRegister(2,name='qr')  # we need a 2-qubit register now
cr = qk.ClassicalRegister(2,name='cr')
cnot_example = qk.QuantumCircuit(qr,cr)

cnot_example.x(qr[0])
cnot_example.cx(qr[0],qr[1]) # First entry is control, the second is the target
cnot_example.measure(qr[0],cr[0])
cnot_example.measure(qr[1],cr[1])

cnot_example.draw(output='mpl')

In [ ]:
job_cnot = qk.execute(cnot_example, backend, shots=1024)
result_cnot = job_cnot.result()
counts_cnot = result_cnot.get_counts()
print(counts_cnot)

<p style='text-align: justify;'> As it happens for the Hadamard gate, also the $\mathrm{CNOT}$ is the inverse of itself, as $\mathrm{CNOT}^2 = \mathcal{I}$. Notice that the role of _target_ and _control_ is not symmetric in the operation carried out by the gate. This is of crucial importance when working with real processors, such as in the IBMQ public devices, since it might happen that, due to hardware limitations, ony a particular direction (i.e. a particular _control_-_target_) configuration can be chosen for a specific pair of qubits. Fortunately, the two opposite choices of target and control are related by simple single-qubit $\mathrm{H}$ rotations. Indeed, you can convince yourself with a simple calculation (and with the output of the qiskit simulation) that the following circuit is completely equivalent to the `cnot_example` above:</p>

In [ ]:
cnot_reversed = qk.QuantumCircuit(qr,cr)

cnot_reversed.x(qr[0])
# This part uses cx(qr[1],qr[0]) but is equivalent to cx(qr[0],qr[1])
cnot_reversed.h(qr[0])
cnot_reversed.h(qr[1])
cnot_reversed.cx(qr[1],qr[0])
cnot_reversed.h(qr[0])
cnot_reversed.h(qr[1])
#
cnot_reversed.measure(qr[0],cr[0])
cnot_reversed.measure(qr[1],cr[1])

cnot_reversed.draw(output='mpl')

In [ ]:
job_cnot_rev = qk.execute(cnot_reversed, backend, shots=1024)
result_cnot_rev = job_cnot_rev.result()
counts_cnot_rev = result_cnot_rev.get_counts()
print(counts_cnot_rev)

<p style='text-align: justify;'> The best way to show the potential of the $\mathrm{CNOT}$ in terms of entanglement generation is to show how it can be useful to generate, starting from the factorized states in the computational basis, the well known set of maximally entangled Bell states. The latter are usually written as
$$
\left|\phi^\pm\right\rangle = \frac{\left|00\right\rangle\pm\left|11\right\rangle}{\sqrt{2}} \qquad \left|\psi^\pm\right\rangle = \frac{\left|01\right\rangle\pm\left|10\right\rangle}{\sqrt{2}}
$$
Now, if $\mathrm{U}_{Bell}$ is the following unitary transformation involving the $\mathrm{CNOT}$ gate between a control qubit $q_c$ and a target qubit $q_t$ 
$$
\mathrm{U}_{Bell} = \mathrm{CNOT}(q_c,q_t)\left(\mathrm{H}_{q_c}\otimes\mathcal{I}_{q_t}\right)
$$
then it is easy to see that
$$
\left|\phi^+\right\rangle = \mathrm{U}_{Bell}\left|00\right\rangle \qquad \left|\phi^-\right\rangle = \mathrm{U}_{Bell}\left|10\right\rangle \qquad \left|\psi^+\right\rangle = \mathrm{U}_{Bell}\left|01\right\rangle \qquad \left|\psi^-\right\rangle = \mathrm{U}_{Bell}\left|11\right\rangle
$$
In qiskit, this is a circuit generating $\left|\phi^+\right\rangle$ from $\left|00\right\rangle$:
</p>

In [ ]:
bell_phi_p = qk.QuantumCircuit(qr,cr)

bell_phi_p.h(qr[0])
bell_phi_p.cx(qr[0],qr[1])
bell_phi_p.measure(qr[0],cr[0])
bell_phi_p.measure(qr[1],cr[1])

bell_phi_p.draw(output='mpl')

In [ ]:
job_bell_phi_p = qk.execute(bell_phi_p, backend, shots=1024)
result_bell_phi_p = job_bell_phi_p.result()
counts_bell_phi_p = result_bell_phi_p.get_counts()
print(counts_bell_phi_p)

* <p style='text-align: justify;'> __Exercise 1.5__ Write and run in qiskit the circuits to generate $\left|\phi^-\right\rangle$, $\left|\psi^+\right\rangle$ and $\left|\psi^-\right\rangle$. </p>
* <p style='text-align: justify;'> __Exercise 1.6__ Find a circuit representation of a unitary transformation that generates the computational basis states of two qubits starting from the Bell states, i.e. $\mathrm{U}_{Bell}^{-1}$. Write and run the corresponding circuits in qiskit to test your results (first generate the Bell states, and then bring them back to the computational basis). </p>

#### The $\mathrm{CZ}$ gate

<p style='text-align: justify;'> Another very common and useful two-qubit gate is the so called controlled-$\mathrm{Z}$ gate. As the name itself points out, this is again an operation where one of the two qubits acts as a _control_, while the _target_ qubit receives a $\mathrm{Z}$ rotation if and only if the state of the control is $\left|1\right\rangle$. Mathematically speaking, the action on the computational basis states is therefore
$$
\mathrm{CZ}\left|x\right\rangle\left|y\right\rangle = \left|x\right\rangle \otimes \mathrm{Z}^x\left|y\right\rangle \qquad x,y \in \{0,1\}
$$
which in matrix notation becomes
$$
\mathrm{CZ} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & -1
\end{pmatrix}
$$
In practice, the net effect is to add a factor $-1$ (more formally, a $\pi$ phase, since $e^{i\pi}=-1$) in front of the $\left|11\right\rangle$ component of the computational basis. Even though the $\mathrm{CZ}$ gate is not directly implemented on the IBMQ processors, you can call it with a clean syntax in qiskit
</p>

In [ ]:
cz_example = qk.QuantumCircuit(qr)

cz_example.cz(qr[0],qr[1])

cz_example.draw(output='mpl')

<p style='text-align: justify;'> At the hardware level, and any time you want to trade a $\mathrm{CZ}$ for a $\mathrm{CNOT}$, the following identity can then be used:
$$
\mathrm{CZ} = \left(\mathcal{I}_{q_c}\otimes\mathrm{H}_{q_t}\right)\mathrm{CNOT}(q_c,q_t)\left(\mathcal{I}_{q_c}\otimes\mathrm{H}_{q_t}\right)
$$
</p>

In [ ]:
cz_from_cnot = qk.QuantumCircuit(qr)

cz_from_cnot.h(qr[1])
cz_from_cnot.cx(qr[0],qr[1])
cz_from_cnot.h(qr[1])

cz_from_cnot.draw(output='mpl')

<p style='text-align: justify;'> An interesting property of the $\mathrm{CZ}$ is that the role of _control_ and _target_ is perfectly symmetric, contrary to the $\mathrm{CNOT}$ case. This fact, which is reflected also in the circuital representation of $\mathrm{CZ}$, is easily seen by combining the above decomposition of $\mathrm{CZ}$ in $\mathrm{CNOT}$ and $\mathrm{H}$ gates with the rule for $\mathrm{CNOT}$ _control_-_target_ reversal and with the fact that $\mathrm{H}^2 = \mathcal{I}$. Finally, it is also imediate to prove by direct calculation that $\mathrm{CZ}^2 = \mathcal{I}$
</p>

* <p style='text-align: justify;'> __Exercise 1.7__ Find a way to obtain $\mathrm{CNOT}$ from $\mathrm{CZ}$ and $\mathrm{H}$ gates. Test all the relations described above between $\mathrm{CNOT}$ and $\mathrm{CZ}$ in qiskit.

#### The $\mathrm{CPHASE}(\delta)$ gate

<p style='text-align: justify;'> This is an extension of the $\mathrm{CZ}$ gate presented above to arbitrary phase rotations. It depends on a continuous parameter $\delta$ and in matrix form reads
$$
\mathrm{CPHASE}(\delta) = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{i\delta}
\end{pmatrix}
$$
Qiskit has a useful shorthand notation for this gate as well (remember that on IBMQ platforms the _phase_ gate is called $\mathrm{U}_1$)
</p>

In [ ]:
delta = 0.5
cphase_test = qk.QuantumCircuit(qr)

cphase_test.cu1(delta,qr[0],qr[1])

cphase_test.draw(output='mpl')

<p style='text-align: justify;'> In terms of the native gates $\mathrm{CNOT}$ and $\mathrm{U}_1$, a possible equivalent circuit implementing the $\mathrm{CPHASE}(\delta)$ is the following: </p>

In [ ]:
delta = 0.5
cphase_from_cnot = qk.QuantumCircuit(qr)

cphase_from_cnot.u1(delta/2,qr[1])
cphase_from_cnot.cx(qr[0],qr[1])
cphase_from_cnot.u1(-delta/2,qr[1])
cphase_from_cnot.cx(qr[0],qr[1])
cphase_from_cnot.u1(delta/2,qr[0])

cphase_from_cnot.draw(output='mpl')

#### The $\mathrm{SWAP}$ gate

<p style='text-align: justify;'> This gate is used to swap the values of two qubits. The action on the computational basis is
$$
\left|x\right\rangle\left|y\right\rangle \rightarrow \left|y\right\rangle\left|x\right\rangle \qquad x,y \in \{0,1\}
$$
and the corresponding matrix representation reads
$$
\mathrm{SWAP} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}
$$
Notice that the action of the gate is actually non-trivial only on the components of the computational basis with different states for the two qubits. In qiskit, you can use this gate with the syntax `.swap` as in the following example:
</p>

In [ ]:
swap_test = qk.QuantumCircuit(qr)

swap_test.swap(qr[0],qr[1])

swap_test.draw(output='mpl')

<p style='text-align: justify;'> An equivalent construction using only $\mathrm{CNOT}$ gates is also possible: </p>

In [ ]:
swap_from_cnot = qk.QuantumCircuit(qr)

swap_from_cnot.cx(qr[0],qr[1])
swap_from_cnot.cx(qr[1],qr[0])
swap_from_cnot.cx(qr[0],qr[1])

swap_from_cnot.draw(output='mpl')

#### Other controlled operations

<p style='text-align: justify;'> In general, any unitary transformation $\mathrm{U}$ on a single qubit can be controlled by the state of another qubit, thus resulting in a $\mathrm{CU}$ two-qubit gate acting as follows:
$$
\mathrm{CU}\left|x\right\rangle\left|y\right\rangle = \left|x\right\rangle \otimes \mathrm{U}^x\left|y\right\rangle \qquad x,y \in \{0,1\}
$$
There always exist decompositions of $\mathrm{CU}$ gates into simpler controlled operations such as the $\mathrm{CNOT}$, possibly with the help of additional single qubit operations. We have already seen some examples with the gates presented above. Qiskit provides directly some useful notation to add other common controlled operations to your circuits: 
</p>

In [ ]:
qr_many = qk.QuantumRegister(5)
control_example = qk.QuantumCircuit(qr_many)

control_example.cy(qr_many[0],qr_many[1])
control_example.ch(qr_many[1],qr_many[2])
control_example.crz(0.2,qr_many[2],qr_many[3])
control_example.cu3(0.5, 0, 0, qr_many[3],qr_many[4])

control_example.draw(output='mpl')

* <p style='text-align: justify;'> __Exercise 1.8__ The $\sqrt{\mathrm{SWAP}}$ is a two-qubit gate with the following matrix expression:
$$
\sqrt{\mathrm{SWAP}} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & (1+i)/2 & (1-i)/2 & 0 \\
0 & (1-i)/2 & (1+i)/2 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}
$$
Find an equivalent circuit implementing this operation using only $\mathrm{CNOT}$ and single qubit gates. Test your predictions with qiskit.
</p>
* <p style='text-align: justify;'> __Exercise 1.9__ Many of the example circuits shown in the two-qubit gates section to present qiskit syntax have trivial measurement outcomes, due to the fact that the initial state is always $\left|00\right\rangle$. Explore the actual effect of any of the two-qubit gates discussed in the paragraphs above by prepending suitable single (and/or other two-qubit gates) transformations. Show and discuss your results in qiskit.  </p>

### BONUS Track
## Three qubits: the Toffoli gate

<p style='text-align: justify;'> Before concluding this section on quantum gates, let us consider a final example involving a three-qubit gate, known as the Toffoli gate. This is a generalization of the $\mathrm{CNOT}$ seen above to the case of two _control_ qubit and one _target_ qubit. The Toffoli gate flips the target qubit (i.e. applies a $\mathrm{X}$ gate) if and only if _both_ controls are in $\left|1\right\rangle$. The usual simbolic notation for the Toffoli gate is therefore $\mathrm{C}^2\mathrm{X}$ or $\mathrm{CCX}$. In mathematical terms, its action on the computational basis is:
$$
\mathrm{CCX}\left|x\right\rangle\left|y\right\rangle\left|z\right\rangle = \left|x\right\rangle\left|y\right\rangle \otimes \mathrm{X}^{x\cdot y}\left|z\right\rangle = \left|x\right\rangle\left|y\right\rangle \left|z\oplus x\cdot y \right\rangle\qquad x,y \in \{0,1\}
$$
More general multi-controlled gates can also be defined (usually denoted $\mathrm{C}^n\mathrm{U}$ for $n$ qubit controling the application of a unitary $\mathrm{U}$ to the target), but they can always be reduced to combinations of single and two qubit gates belonging to a universal set. The Toffoli itself can be obtained by a a combination of elementary gates, as shown in the circuits below:
</p>

In [ ]:
qrthree = qk.QuantumRegister(3)
toffoli_example = qk.QuantumCircuit(qrthree)

toffoli_example.ccx(qrthree[0],qrthree[1],qrthree[2])

toffoli_example.draw(output='mpl')

In [ ]:
toffoli_from_cnot = qk.QuantumCircuit(qrthree)

toffoli_from_cnot.h(qrthree[2])
toffoli_from_cnot.cx(qrthree[1],qrthree[2])
toffoli_from_cnot.tdg(qrthree[2])
toffoli_from_cnot.cx(qrthree[0],qrthree[2])
toffoli_from_cnot.t(qrthree[2])
toffoli_from_cnot.cx(qrthree[1],qrthree[2])
toffoli_from_cnot.tdg(qrthree[2])
toffoli_from_cnot.cx(qrthree[0],qrthree[2])
toffoli_from_cnot.tdg(qrthree[1])
toffoli_from_cnot.t(qrthree[2])
toffoli_from_cnot.h(qrthree[2])
toffoli_from_cnot.cx(qrthree[0],qrthree[1])
toffoli_from_cnot.tdg(qrthree[1])
toffoli_from_cnot.cx(qrthree[0],qrthree[1])
toffoli_from_cnot.s(qrthree[1])
toffoli_from_cnot.t(qrthree[0])

toffoli_from_cnot.draw(output='mpl')

## Final exercise

* <p style='text-align: justify;'> __Exercise 1.11__ Run as many of the examples in this tutorial as you can on IBMQ quantum processors, adapting the circuits whenever needed or possible and choosing the most appropriate mapping on the physical qubits. For simple circuits, you may also use the composer, a graphical quantum programming interface available at <https://quantumexperience.ng.bluemix.net/qx/editor>. </p>